#### Implementação baseada em uma interface intermediária:

![Memento](Memento.png)

1. A classe __Originator__ pode produzir retratos de seu próprio estado, bem como restaurar seu estado de retratos quando necessário.
   _Cria e restaura o estado._

2. O __Memento__ é um objeto de valor que age como um retrato do estado da originadora. É uma prática comum fazer o memento imutável e passar os dados para ele apenas uma vez, através do construtor.
   _Armazena a cópia do estado do Originator_

3. A __Caretaker__ sabe não só “quando” e “por quê” capturar o estado da originadora, mas também quando o estado deve ser restaurado.
   
   Uma cuidadora pode manter registros do histórico da originadora armazenando os mementos em um pilha. Quando a originadora precisa voltar atrás no histórico, a cuidadora busca o memento mais do topo da pilha e o passa para o método de restauração da originadora.

In [19]:
from __future__ import annotations
from copy import deepcopy

In [20]:
class Memento:

    """ Memento """

    def __init__(self, state: dict) -> None:
        self._state: dict
        
        # Passagem de dados através do construtor
        object.__setattr__(self, '_state', state) 

    def get_state(self) -> dict:
        return self._state
    
    def __setattr__(self, name, value):
        """
        É uma prática comum fazer o memento imutável e passar os 
        dados para ele apenas uma vez, através do construtor.
        """
        raise AttributeError('Sorry, I am Immutable!')

In [21]:
class ImageEditor:

    """ Originator """

    def __init__(self, name: str, width: int, height: int) -> None:
        self.name = name
        self.width = width
        self.height = height

    def save_state(self) -> Memento:
        return Memento(deepcopy(self.__dict__)) 

    def restore(self, memento: Memento) -> None:
         self.__dict__ = memento.get_state()

    def __str__(self) -> str:
        return f'{self.__class__.__name__}: ({self.__dict__})'

In [22]:
class Caretaker:

    """ Caretaker """

    def __init__(self, originator: ImageEditor):
        self._originator = originator
        self._mementos: list[Memento] = []

    def backup(self) -> None:
        self._mementos.append(self._originator.save_state())

    def restore(self) -> None:
        if not self._mementos:
            return
        
        self._originator.restore(self._mementos.pop())

In [27]:
if __name__ == "__main__":

    img = ImageEditor('Image1.jpg', 600, 300)

    caretaker = Caretaker(img)

    # Backups
    print('\ngenerating backup:')
    
    caretaker.backup()
    print(img)

    img.name = 'Image2.jpg'
    img.height = 400
    img.width = 200
    caretaker.backup()
    print(img)

    img.name = 'Image3.jpg'
    img.height = 100
    img.width = 50
    caretaker.backup()
    print(img)

    img.name = 'Image4.jpg'
    img.height = 150
    img.width = 500
    caretaker.backup()
    print(img)

    # restores
    print('\nrestoring backups:')

    caretaker.restore()
    print(img)

    caretaker.restore()
    print(img)

    caretaker.restore()
    print(img)

    caretaker.restore()
    print(img)


generating backup:
ImageEditor: ({'name': 'Image1.jpg', 'width': 600, 'height': 300})
ImageEditor: ({'name': 'Image2.jpg', 'width': 200, 'height': 400})
ImageEditor: ({'name': 'Image3.jpg', 'width': 50, 'height': 100})
ImageEditor: ({'name': 'Image4.jpg', 'width': 500, 'height': 150})

restoring backups:
ImageEditor: ({'name': 'Image4.jpg', 'width': 500, 'height': 150})
ImageEditor: ({'name': 'Image3.jpg', 'width': 50, 'height': 100})
ImageEditor: ({'name': 'Image2.jpg', 'width': 200, 'height': 400})
ImageEditor: ({'name': 'Image1.jpg', 'width': 600, 'height': 300})
